# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,salinopolis,-0.6136,-47.3561,25.57,88,31,4.32,BR,1720147662
1,1,yellowknife,62.4560,-114.3525,17.29,67,75,4.63,CA,1720147424
2,2,wanlaweyn,2.6185,44.8938,23.32,85,90,4.66,SO,1720147670
3,3,port mathurin,-19.6833,63.4167,22.62,60,85,9.99,MU,1720147671
4,4,iqaluit,63.7506,-68.5145,3.85,93,100,2.06,CA,1720147672


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & 
                    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
new_city_data_df.dropna()

# Display sample data
new_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,montrose,38.4166,-108.2673,21.76,21,0,1.94,US,1720147700
45,45,waddan,29.1614,16.1390,23.31,64,0,1.15,LY,1720147722
59,59,albufeira,37.0882,-8.2503,26.91,41,0,3.55,PT,1720147739
71,71,ain sukhna,29.6002,32.3167,26.98,59,0,2.45,EG,1720147753
100,100,miracema do tocantins,-9.5618,-48.3967,23.35,41,0,1.95,BR,1720147789


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
list_of_columns = ['City', 'Country','Lat', 'Lng', 'Humidity']
hotel_df = new_city_data_df[list_of_columns].copy().reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,26,montrose,US,38.4166,-108.2673,21,
1,45,waddan,LY,29.1614,16.1390,64,
2,59,albufeira,PT,37.0882,-8.2503,41,
3,71,ain sukhna,EG,29.6002,32.3167,59,
4,100,miracema do tocantins,BR,-9.5618,-48.3967,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
montrose - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
albufeira - nearest hotel: Hotel Baltum
ain sukhna - nearest hotel: العين السخنة
miracema do tocantins - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
nova vicosa - nearest hotel: Pousada Pontal da Barra
salina - nearest hotel: Baymont
itaueira - nearest hotel: No hotel found
powell river - nearest hotel: Beyond Bliss Suites & Spa
kuhdasht - nearest hotel: هتل ازادی
chusovoy - nearest hotel: Чусовская
tripoli - nearest hotel: الاصيل
mao - nearest hotel: Hevresac
sao miguel do araguaia - nearest hotel: Hotel São Miguel
corbin - nearest hotel: Corbin Inn
ramtown - nearest hotel: Crystal Point Inn
awjilah - nearest hotel: No hotel found


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,26,montrose,US,38.4166,-108.2673,21,No hotel found
1,45,waddan,LY,29.1614,16.1390,64,No hotel found
2,59,albufeira,PT,37.0882,-8.2503,41,Hotel Baltum
3,71,ain sukhna,EG,29.6002,32.3167,59,العين السخنة
4,100,miracema do tocantins,BR,-9.5618,-48.3967,41,No hotel found
5,107,westport,US,41.1415,-73.3579,83,Norwalk Inn and Conference Center
6,118,nova vicosa,BR,-17.8919,-39.3719,77,Pousada Pontal da Barra
7,218,salina,US,38.8403,-97.6114,63,Baymont
8,247,itaueira,BR,-7.6033,-43.0256,48,No hotel found
9,254,powell river,CA,49.8496,-124.5361,53,Beyond Bliss Suites & Spa


In [22]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)